In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from numpy.random import seed

import re
import sklearn

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
from pandas.io.json import json_normalize 
data = pd.read_csv('/Users/Sandi/Desktop/Projects/Capstone_2/questionnaire.csv')
data.head()

,SEQN,ACD011A,ACD011B,ACD011C,ACD040,ACD110,ALQ101,ALQ110,ALQ120Q,ALQ120U,...,WHD080U,WHD080L,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ030M,WHQ500,WHQ520
0,73557,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,3.0,...,NaN,40.0,270.0,200.0,69.0,270.0,62.0,NaN,NaN,NaN
1,73558,1.0,NaN,NaN,NaN,NaN,1.0,NaN,7.0,1.0,...,NaN,NaN,240.0,250.0,72.0,250.0,25.0,NaN,NaN,NaN
2,73559,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,...,NaN,NaN,180.0,190.0,70.0,228.0,35.0,NaN,NaN,NaN
3,73560,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0
4,73561,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,...,NaN,NaN,150.0,135.0,67.0,170.0,60.0,NaN,NaN,NaN


In [3]:
demographics = pd.read_csv('/Users/Sandi/Desktop/Projects/Capstone_2/demographic.csv')
demographics.head()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,73557,8,2,1,69,NaN,4,4,1.0,NaN,...,3.0,4.0,NaN,13281.237386,13481.042095,1,112,4.0,4.0,0.84
1,73558,8,2,1,54,NaN,3,3,1.0,NaN,...,3.0,1.0,1.0,23682.057386,24471.769625,1,108,7.0,7.0,1.78
2,73559,8,2,1,72,NaN,3,3,2.0,NaN,...,4.0,1.0,3.0,57214.803319,57193.285376,1,109,10.0,10.0,4.51
3,73560,8,2,1,9,NaN,3,3,1.0,119.0,...,3.0,1.0,4.0,55201.178592,55766.512438,2,109,9.0,9.0,2.52
4,73561,8,2,2,73,NaN,3,3,1.0,NaN,...,5.0,1.0,5.0,63709.667069,65541.871229,2,116,15.0,15.0,5.00


**Data Set Merge and Select Attribute**

In [4]:
data.drop(['SEQN'], axis = 1, inplace=True)
demographics.drop(['SEQN'], axis = 1, inplace=True)

#https://www.kaggle.com/what0919/diabetes-prediction

In [5]:
df = pd.concat([data, demographics], axis=1, join='inner')
df.describe()
#merge

,ACD011A,ACD011B,ACD011C,ACD040,ACD110,ALQ101,ALQ110,ALQ120Q,ALQ120U,ALQ130,...,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
count,5759.0,16.0,171.0,2374.000000,1007.000000,5421.000000,1631.000000,4479.000000,3593.000000,3596.000000,...,9881.000000,10052.000000,5342.000000,10175.000000,10175.000000,10175.000000,10175.000000,10042.000000,10052.000000,9390.000000
mean,1.0,8.0,9.0,3.101095,2.956306,1.311197,1.594727,4.709310,1.921514,3.511123,...,3.502378,2.735078,3.612879,30585.180933,30585.180933,1.484128,110.926880,10.884983,10.505969,2.252153
std,0.0,0.0,0.0,1.511821,1.733794,0.545023,0.615303,34.428362,0.853701,28.866122,...,1.251080,5.922690,1.321508,26948.427403,27941.005388,0.499773,4.260322,13.878864,13.643064,1.634907
min,1.0,8.0,9.0,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,3697.772098,0.000000,1.000000,104.000000,1.000000,1.000000,0.000000
25%,1.0,8.0,9.0,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,1.000000,3.000000,12754.485277,12561.598665,1.000000,107.000000,5.000000,5.000000,0.870000
50%,1.0,8.0,9.0,3.000000,3.000000,1.000000,2.000000,2.000000,2.000000,2.000000,...,4.000000,1.000000,4.000000,20232.999616,20174.573544,1.000000,111.000000,8.000000,7.000000,1.705000
75%,1.0,8.0,9.0,4.000000,5.000000,2.000000,2.000000,4.000000,3.000000,3.000000,...,4.000000,4.000000,5.000000,36280.427224,36748.217798,2.000000,115.000000,14.000000,14.000000,3.607500
max,1.0,8.0,9.0,9.000000,5.000000,9.000000,9.000000,999.000000,3.000000,999.000000,...,9.000000,99.000000,9.000000,167884.543709,171395.264901,2.000000,118.000000,99.000000,99.000000,5.000000


**NaN Handling and Feature Column Selection**

In [19]:
from sklearn.feature_selection import VarianceThreshold

df.dropna(axis=1, how='all')
df.dropna(axis=0, how='all')

,ACD011A,ACD011B,ACD011C,ACD040,ACD110,ALQ101,ALQ110,ALQ120Q,ALQ120U,ALQ130,...,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,3.0,1.0,...,3.0,4.0,NaN,13281.237386,13481.042095,1,112,4.0,4.0,0.84
1,1.0,NaN,NaN,NaN,NaN,1.0,NaN,7.0,1.0,4.0,...,3.0,1.0,1.0,23682.057386,24471.769625,1,108,7.0,7.0,1.78
2,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,...,4.0,1.0,3.0,57214.803319,57193.285376,1,109,10.0,10.0,4.51
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,4.0,55201.178592,55766.512438,2,109,9.0,9.0,2.52
4,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,...,5.0,1.0,5.0,63709.667069,65541.871229,2,116,15.0,15.0,5.00
5,NaN,NaN,NaN,4.0,NaN,1.0,NaN,5.0,3.0,1.0,...,4.0,3.0,NaN,24978.144602,25344.992359,1,111,9.0,9.0,4.79
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,1.0,5.0,18040.818432,17913.555067,1,105,15.0,15.0,5.00
7,1.0,NaN,NaN,NaN,NaN,2.0,1.0,2.0,3.0,1.0,...,5.0,2.0,NaN,60325.095254,61758.654880,1,114,10.0,10.0,5.00
8,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,5.0,23307.675629,0.000000,2,106,15.0,15.0,5.00
9,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,...,3.0,3.0,NaN,16351.783140,17480.115175,1,112,4.0,4.0,0.48


In [6]:
#https://wwwn.cdc.gov/Nchs/Nhanes/Search/variablelist.aspx?Component=Questionnaire&CycleBeginYear=2013
##The variable dictionary to understand the code at the top of each column.  

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10175 entries, 0 to 10174
Columns: 952 entries, ACD011A to WHQ520
dtypes: float64(944), int64(6), object(2)
memory usage: 73.9+ MB


Medical Conditions  MCQ084 
difficulties in thinking or remembering that can make a big difference in everyday activities.
The next question asks about difficulties in thinking or remembering that can make a big difference in everyday activities. This does not refer to occasionally forgetting your keys or the name of someone you recently met. This refers to things like confusion or memory loss that are happening more often or getting worse. We want to know how these difficulties impact {you/SP}. During the past 12 months, {have you/has she/has he} experienced confusion or memory loss that is happening more often or is getting worse?

In [8]:
data.MCQ084.value_counts()

2.0    1521
1.0     318
9.0       1
Name: MCQ084, dtype: int64

In [9]:
data.MCQ084.count()

1840

https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/MCQ_H.htm#MCQ084
- 1521 is the count of No
- 318 is the count of Yes
- 1 is the count of don't know
- 7,930 are missing

In [10]:
data.MCQ084.tail(10)

10165    NaN
10166    2.0
10167    1.0
10168    NaN
10169    NaN
10170    NaN
10171    NaN
10172    NaN
10173    NaN
10174    NaN
Name: MCQ084, dtype: float64

In [11]:
#drop the NaN values.  How many rows re
memory_difficulty= data.MCQ084.dropna(axis = 0)
memory_difficulty.tail(10)

#axis=0  row   axis = 1  column

10127    2.0
10130    2.0
10135    2.0
10145    2.0
10151    2.0
10155    2.0
10160    2.0
10161    1.0
10166    2.0
10167    1.0
Name: MCQ084, dtype: float64

In [12]:
memory_difficulty.count()

1840

https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/PAQ_H.htm
-Past wk # days cardiovascular exercise  Physical Activity PAQ677
the first column is the number od days, the second column is the count
99 represents "I don't know"
-8805 are missing

Physical Activity PAQ677 On how many of the past 7 days did {you/SP} exercise or participate in physical activity for at least 20 minutes that made {you/him/her} sweat and breathe hard, such as basketball, soccer, running, swimming laps, fast bicycling, fast dancing, or similar activities?

In [13]:
data.PAQ677.value_counts()

5.0     154
0.0     111
3.0     110
2.0      81
4.0      65
1.0      62
7.0      52
6.0      43
99.0      1
Name: PAQ677, dtype: int64

In [14]:
data.PAQ677.count()

679

-Past wk # days strengthened muscles Physical Activity PAQ678
the first column is the number od days, the second column is the count
-8805 are missing

Physical Activity PAQ678 On how many of the past 7 days did {you/SP} do exercises to strengthen or tone {your/his/her} muscles, such as push-ups, sit-ups, or weight lifting?

In [15]:
data.PAQ678.value_counts()

0.0    207
2.0    109
3.0    106
1.0     96
5.0     77
4.0     39
7.0     32
6.0     13
Name: PAQ678, dtype: int64

In [16]:
data.PAQ678.count()

679

-Days physically active at least 60 min.
-6495 are missing
first column is number of days.
99 represents the don't know, which in this set was 8.

Physical Activity PAQ706 During the past 7 days, on how many days {were you/was SP} physically active for a total of at least 60 minutes per day? Add up all the time {you/he/she} spent in any kind of physical activity that increased {your/his/her} heart rate and made {you/him/her} breathe hard some of the time.

In [17]:
data.PAQ706.value_counts()

7.0     1799
5.0      314
3.0      194
4.0      183
2.0      159
0.0      131
6.0      111
1.0       90
99.0       8
Name: PAQ706, dtype: int64

In [18]:
data.PAQ706.count()

2989